In [2]:
import pandas as pd
import numpy as np
import numpy.typing as npt
from typing import NamedTuple, Callable

In [3]:
Images = npt.NDArray[np.int16]
# Layer = NamedTuple('Layer', [
#     ('weight', npt.NDArray[np.float32]),
#     ('bias', npt.NDArray[np.float32]),
#     ('activation', Callable),
#     ('back_prop', Callable)
# ]
# )


# NN (Perceptron) implementation
- 2 layers (with 10 neurons each)
    - Hidden layer (Size of the amount of pixel it image has )
    - Output layer (indicates one digit from 0 to 9)
- 784 inputs (1 image of 28x28 pixels)


In [4]:
class Layer:
    weight: npt.NDArray[np.float32]
    bias: npt.NDArray[np.float32]
    activation: Callable
    derivate: Callable

    def __init__(self, 
                 weight: npt.NDArray[np.float32],
                 bias: npt.NDArray[np.float32]) -> None:
        
        self.weight = weight
        self.bias = bias

        self.activation = self._ReLU
        self.derivate = self._ReLU_derive
        

    def _ReLU(self, input: np.ndarray) -> np.ndarray:
        return np.maximum(0, input)
    
    def _ReLU_derive(self, z: np.ndarray):
        return z > 0


In [5]:
class NeuralNetwork:

    _layers = list[Layer]
    _inputs: npt.NDArray[np.float32]
    _bias: npt.NDArray[np.float32]

    def _normalize(self, arr: Images) -> npt.NDArray[np.float32]:
        """Normalize a array with N number """
        ...

    def _encode_target(self, target: Images) -> npt.NDArray[np.int8]:
        """One hot encoding of the target predictions"""
        enc = np.zeros((target.size, target.max() + 1)) # array of N labels
        enc[np.arange(target.size), target] = 1 # insert 1 in the position that represents the label
        return enc.astype(np.int8).T # As coluns


    def __init__(self, inputs: Images, target: Images) -> None:

        self._inputs = self._normalize(inputs)
        self._targets = self._encode_target(target)
        self._layers = []

    def add_layer(self, neurons_size: int ,inputs_size: int, actvation: Callable, back: Callable):
        weight = np.random.rand(neurons_size, inputs_size).astype('float32')
        bias = np.random.rand(neurons_size, 1).astype('float32')
        layer = Layer(weight, bias)
        self._layers.append(layer)

    def foward_propagation(self, layer: Layer, input: np.ndarray):
        layer.z  = layer.weight.dot(input) + layer.bias
        layer.a = layer.activation(z)
        return


# Aux Functions

In [ ]:
def get_train_test_data(data: Images, test_size: int) -> tuple[tuple[Images, Images], tuple[Images, Images]]:
    """Split the data randomly into train and test data"""
    data = data.copy()
    rows, cols = data.shape
    np.random.shuffle(data)
    
    data_test = data[0: test_size].T
    x_test = data_test[1: cols]
    y_test = data_test[0]

    data_train = data[1000: rows].T
    x_train = data_train[1: cols]
    y_train = data_train[0]

    # Split data into labels and input data
    return (x_train, y_train), (x_test, y_test)


# Import dataset's with the pixels

In [ ]:
data = np.array(pd.read_csv('./digit-recognizer/train.csv', sep=',').astype('int16'))
test = pd.read_csv('./digit-recognizer/test.csv')

# Transponding dataset so each column is a example image

- Also shuffling the dataset (Prevent overfitting while spliting between train and test)

In [ ]:
(x_train, y_train), (x_test, y_test) = get_train_test_data(data, 1000)


In [ ]:
np.random.rand(NeuralNetwork.NEURONS_SIZE, x_train.shape[0]).max()


0.9999685329440432